<h1><center>Лабораторна робота 4.</center></h1>
<h2><center>Виявлення сарказму за допомогою логістичної регресії</center></h2>

**Виконав:** Прізвище І.П.

**Варіант:** №__

В цій лабораторній роботі ми будемо подувати модель логістичної регресії для розв'язання задачі класифікації саркастичних текстів. Для цього використаємо набір даних з [наукової статі](https://arxiv.org/abs/1704.05579) «A Large Self-Annotated Corpus for Sarcasm» з понад 1 мільйоном коментарів із платформи Reddit, що позначені авторами статті як саркастичні або ні. Оброблена версія цього набору даних міститься на платформі Kaggle у формі [Набору даних Kaggle](https://www.kaggle.com/datasets/danofer/sarcasm/data).

<a class="anchor" id="lab-4"></a>

## Зміст

- [4.1. Попередній аналіз даних](#lab-4.1)
- [4.2. Векторизація TF-IDF та логістична регресія](#lab-4.2)
- [4.3. Інтерпретація та порівняння моделі](#lab-4.3)
- [4.4. Розширене вдосконалення моделі](#lab-4.4)
- [4.5. Практичне застосування результатів інтелектуального аналізу даних](#lab-4.5)

Спершу завантажте цільовий набір даних `train-balanced-sarcasm.csv` із ресурсу Kaggle за [посиланням](https://www.kaggle.com/datasets/danofer/sarcasm/data?select=train-balanced-sarcasm.csv). Збережіть файл .csv в теку з робочим файлом лабораторної роботи.

Цей набір даних містить 1,3 млн. саркастичних коментарів з платформи інтернет-коментарів Reddit. Набір даних було створено шляхом вилучення коментарів з Reddit, що містять тег `\s` ("сарказм"). Цей тег переважно використовується користувачами, щоб вказати на жартівливий тон їхнього коментаря, тобто такий запис не повинен сприйматися серйозно, і, як правило, є надійним показником саркастичного змісту коментаря.

Дані мають збалансовану та незбалансовану (тобто справжній розподіл) версії. Реальне співвідношення несаркастичних до саркастичних коментарів становить приблизно 1:100. Оригінальний набір даних можна додатково отримати за [посиланням](https://github.com/NLPrinceton/SARC), але для лабораторної роботи він нам не потрібен.


In [1]:
PATH_TO_DATA = "train-balanced-sarcasm.csv"

In [2]:
# Виконаємо завантаження основних бібліотек
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [3]:
train_df = pd.read_csv(PATH_TO_DATA)

Нижче переглянемо зразки цільового набору в розрізі основних ознак.

In [4]:
train_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   label           1010826 non-null  int64 
 1   comment         1010773 non-null  object
 2   author          1010826 non-null  object
 3   subreddit       1010826 non-null  object
 4   score           1010826 non-null  int64 
 5   ups             1010826 non-null  int64 
 6   downs           1010826 non-null  int64 
 7   date            1010826 non-null  object
 8   created_utc     1010826 non-null  object
 9   parent_comment  1010826 non-null  object
dtypes: int64(4), object(6)
memory usage: 77.1+ MB


Деяких коментарів бракує в наборі, тому ми видалимо відповідні рядки.

In [6]:
train_df.dropna(subset=["comment"], inplace=True)

Переконаємось, що розглядуваний набір даних є справді збалансованим.

In [7]:
train_df["label"].value_counts()

0    505405
1    505368
Name: label, dtype: int64

Розділимо дані на частини для навчання та валідації.

In [8]:
train_texts, valid_texts, y_train, y_valid = train_test_split(
    train_df["comment"], train_df["label"], random_state=17
)

<a class="anchor" id="lab-4.1"></a>

## <span style="color:blue; font-size:1.2em;">4.1. Попередній аналіз даних</span>

[Повернутися до змісту](#lab-4)

In [9]:
# from wordcloud import STOPWORDS, WordCloud

In [10]:
# Підготуємо об'єкт класу WordCloud для 
# можливого візуального аналізу  
# my_wordcloud = WordCloud(
#     background_color="black",
#     stopwords=STOPWORDS,
#     max_words=200,
#     max_font_size=100,
#     random_state=17,
#     width=800,
#     height=400,
# )

<span style="color:red; font-size:2em;">Завдання 1</span>

**Для всіх варіантів:**

У завданні 1 вам потрібно проаналізувати та підготувати набір даних для подальшої побудови моделей інтелектуального аналізу даних.

**Для варіантів 1-2:**

Проаналізуйте використання в текстах хештегів, URLs, емоджі та згадок користувачів (символ - @).

**Для варіантів 3-4:**

Векторизуйте текст на слова. Обрахуйте та візуалізуйте 20 найбільш вживаних слів.

**Для варіантів 5-6:**

Візуалізуйте розподіли саркастичних і нормальних коментарів.

**Для варіантів 7-8:**

Проаналізуйте, чи є деякі субредити в середньому більш «саркастичними», ніж інші.

**Для варіантів 9-10:**

Візуалізуйте хмарне подання слів для кожного класу.

**Для варіантів 11-12:**

Видаліть загальні стоп-слова з текстових даних.

**Для варіантів 13-14:**

Дослідіть та візуалізуйте розподіли довжини тексту для різних класів.

**Для варіантів 15-16:**

Напишіть функцію для очищення текстових даних (видалення спеціальних символів, нижнього регістру тощо).

**Для варіантів 17-18:**

Створіть хмару слів для найуживаніших слів у класі "Саркастичний".

**Для варіантів 19-20:**

Проаналізуйте, чи є деякі автори в середньому більш саркастичними, ніж інші.


<a class="anchor" id="lab-4.2"></a>

## <span style="color:blue; font-size:1.2em;">4.2. Векторизація TF-IDF та логістична регресія</span>

[Повернутися до змісту](#lab-4)

In [11]:
# Побудуємо біграми, обмежимо максимальну кількість ознак
# і мінімальну частоту слів
tf_idf_3 = TfidfVectorizer(ngram_range=(1, 2), max_features=5000, min_df=2)
# Створимо об'єкт мультиноміальної логістичної регресії,
# яка також відома як класифікатор softmax
logit_3 = LogisticRegression(C=1, n_jobs=4, solver="lbfgs", random_state=17, verbose=1)
# Задамо sklearn's pipeline
tfidf_logit_pipeline_3 = Pipeline([("tf_idf", tf_idf), ("logit", logit)])

In [12]:
%%time
tfidf_logit_pipeline.fit(train_texts, y_train)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


CPU times: total: 19.9 s
Wall time: 29.9 s


Pipeline(steps=[('tf_idf',
                 TfidfVectorizer(max_features=50000, min_df=2,
                                 ngram_range=(1, 2))),
                ('logit',
                 LogisticRegression(C=1, n_jobs=4, random_state=17,
                                    verbose=1))])

In [13]:
%%time
valid_pred = tfidf_logit_pipeline.predict(valid_texts)

CPU times: total: 4.17 s
Wall time: 4.61 s


In [14]:
print(f"Точність за даними валідації: {accuracy_score(y_valid, valid_pred): .4f}")

Точність за даними валідації:  0.7210


In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import cProfile
import pstats

# Splitting the dataset
train_texts, valid_texts, y_train, y_valid = train_test_split(
    train_df["comment"], train_df["label"], random_state=17
)

In [16]:
# Function to encapsulate the pipeline execution
def run_pipeline(train_texts, y_train, valid_texts):
    # Setting up the TfidfVectorizer
    tf_idf = TfidfVectorizer(ngram_range=(1, 2), max_features=20000, min_df=5)

    # Creating a Decision Tree Classifier with limited depth
    decision_tree = DecisionTreeClassifier(max_depth=10, random_state=17)

    # Setting up the pipeline
    tfidf_tree_pipeline = Pipeline([("tf_idf", tf_idf), ("decision_tree", decision_tree)])

    # Fitting the model
    tfidf_tree_pipeline.fit(train_texts, y_train)

    # Making predictions
    valid_pred = tfidf_tree_pipeline.predict(valid_texts)
    return valid_pred

In [17]:
# Profiling the pipeline execution
profiler = cProfile.Profile()
profiler.enable()

# Run the pipeline
valid_pred = run_pipeline(train_texts, y_train, valid_texts)

profiler.disable()

# Print the profiling results
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats()

         35353936 function calls (35353926 primitive calls) in 58.268 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   58.268   29.134 C:\Users\radiu\anaconda3\envs\courses\lib\site-packages\IPython\core\interactiveshell.py:3424(run_code)
        2    0.000    0.000   58.268   29.134 {built-in method builtins.exec}
        1    0.146    0.146   58.268   58.268 C:\Users\radiu\AppData\Local\Temp\ipykernel_24332\190850411.py:6(<module>)
        1    0.012    0.012   58.122   58.122 C:\Users\radiu\AppData\Local\Temp\ipykernel_24332\651167869.py:2(run_pipeline)
        1    0.000    0.000   52.676   52.676 C:\Users\radiu\anaconda3\envs\courses\lib\site-packages\sklearn\pipeline.py:374(fit)
        1    0.007    0.007   26.792   26.792 C:\Users\radiu\anaconda3\envs\courses\lib\site-packages\sklearn\tree\_classes.py:859(fit)
        1    0.002    0.002   26.785   26.785 C:\Users\radiu\anaconda3\envs\

In [20]:
my_result = accuracy_score(y_valid, valid_pred)

my_result

0.5903662136813695

<span style="color:red; font-size:2em;">Завдання 2</span>

**Для всіх варіантів:**

У завданні 2 вам потрібно доповнити побудовані на попередньому кроці векторизатор та модель логістичної регресії.

**Для варіантів 1-2:**

Використайте N-грами у векторизаторі TF-IDF та поясніть, як вони працюють.

**Для варіантів 3-4:**

Дослідіть вплив регулярізації L1 (Lasso) і L2 (Ridge) на побудовану модель логістичної регресії. Зробіть висновок щодо впливу певного типу регулярізації (Lasso та Ridge) на результат моделювання.

**Для варіантів 5-6:**

Проведіть експерименти з параметром `max_features` у векторизаторі TF-IDF і зробіть висновок щодо його впливу на якість створених ознак та точність класифікації.

**Для варіантів 7-8:**

Проведіть експерименти з різними пороговими значеннями для моделі логістичної регресії. Дослідіть, як зміна порогу впливає на частку правильних відповідей, яку генерує побудована модель.

**Для варіантів 9-10:**

Застосуйте k-кратну перехресну перевірку (k-fold cross-validation) до побудованої моделі логістичної регресії та порівняйте результати за валідаційним набором. Якими є переваги та недоліки перехресної перевірки?

**Для варіантів 11-12:**

Інтерпретуйте результати моделі логістичної регресії, з огляду на значення коефіцієнтів цієї моделі. Як отримані значення вплинули на результат задачі класифікації?

**Для варіантів 13-14:**

Візуалізуйте та інтерпретуйте криві навчання та валідації побудованої логістичної регресії.

**Для варіантів 15-16:**

Налаштуйте наявні гіперпараметри моделі логістичної регресії за механізмом `GridSearchCV`.

**Для варіантів 17-18:**

Вилучіть та візуалізуйте назви ознак із векторизатора TF-IDF.

**Для варіантів 19-20:**

Проаналізуйте ознаки, які були створені векторизатором TF-IDF. Дослідіть вплив різних слів та їхні відповідні TF-IDF-оцінки.


<a class="anchor" id="lab-4.3"></a>

## <span style="color:blue; font-size:1.2em;">4.3. Інтерпретація та порівняння моделі</span>

[Повернутися до змісту](#lab-4)

In [18]:
# Довідково: Нижче подамо матрицю невідповідностей
def plot_confusion_matrix(
    actual,
    predicted,
    classes,
    normalize=False,
    title="Confusion matrix",
    figsize=(7, 7),
    cmap=plt.cm.Blues,
    path_to_save_fig=None,
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools

    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(actual, predicted).T
    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=figsize)
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = ".2f" if normalize else "d"
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(
            j,
            i,
            format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black",
        )

    plt.tight_layout()
    plt.ylabel("Predicted label")
    plt.xlabel("True label")

    if path_to_save_fig:
        plt.savefig(path_to_save_fig, dpi=300, bbox_inches="tight")

<span style="color:red; font-size:2em;">Завдання 3</span>

**Для всіх варіантів:**

У завданні 3 вам потрібно інтерпретувати отримані результати щодо передбачення саркастичного тексту.

**Для варіантів 1-2:**

Візуалізуйте на одному графіку криві навчання та валідації побудованої регресійної моделі. Порівняйте обидві криві, зробіть висновок щодо якості навченої моделі.

**Для варіантів 3-4:**

Обрахуйте та інтерпретуйте співвідношення шансів для конкретних ознак, з огляду на те як вони пов’язані з можливістю зустріти саркастичний текст.

**Для варіантів 5-6:**

Виконайте візуальний аналіз тих випадків, за яких модель логістичної регресії дала неправильні прогнози, і запропонуйте стратегії для покращення ефективності моделі на основі цього аналізу.

**Для варіантів 7-8:**

Побудуйте модель дерева рішень на тих самих даних. Порівняйте її ефективність із моделлю логістичної регресії та обговоріть плюси та мінуси кожної моделі.

**Для варіантів 9-10:**

Побудуйте графіки часткової залежності ([partial dependence plots (PDP)](https://scikit-learn.org/stable/modules/partial_dependence.html#:~:text=Partial%20dependence%20plots%20(PDP)%20show,the%20'complement'%20features).)) для 5 найбільш значущих ознак у побудованій моделі логістичної регресії. Поясніть, як PDP відображають зв’язок між ознакою та прогнозованим результатом, зберігаючи інші ознаки незмінними. Інтерпретуйте ці графіки для розглядуваної задачі виявлення сарказму.

**Для варіантів 11-12:**

Оцініть побудовану модель логістичної регресії за допомогою кількох статистичних метрик (accuracy, precision, recall, F1-score) та зробіть висновок щодо якості моделі з огляду на значення цих метрик. Для виконання завдання використайте клас [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) бібліотеки scikit-learn.

**Для варіантів 13-14:**

Проаналізуйте та інтерпретуйте ті випадки, які побудована модель логістичної регресії класифікувала неправильно.

**Для варіантів 15-16:**

Побудуйте та інтерпретуйте криві навчання та валідації для побудованої моделі логістичної регресії. Проаналізуйте, що ці криві відображають з огляду на ефективність моделі та її можливе недонавчання або перенавчання.

**Для варіантів 17-18:**

Виконайте візуальний аналіз помилок побудованої моделі. Запропонуйте та реалізуйте стратегії для покращення моделі та переобрахуйте її для розв'язання задачі виявлення саркастичних текстів.

**Для варіантів 19-20:**

Побудуйте модель лінійної регресії на тих самих даних, перетворивши цільову змінну за необхідності. Порівняйте ефективність моделі лінійної регресії із моделлю логістичної регресії та обговоріть, чому лінійна регресія може бути непридатною для цього завдання.


<a class="anchor" id="lab-4.4"></a>

## <span style="color:blue; font-size:1.2em;">4.4. Розширене вдосконалення моделі</span>

[Повернутися до змісту](#lab-4)

<span style="color:red; font-size:2em;">Завдання 4</span>

**Для всіх варіантів:**

У завданні 4 вам потрібно використати різноманітні програмні механізми для покращення роботи моделі.

**Для варіантів 1-2:**

Побудуйте векторизатори TF-IDF окремо для коментарів і субредітів. Далі об’єднайте ці новостворені ознаки разом та побудуйте нову модель логістичної регресії з використанням [sklearn.pipeline.Pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). Порівняйте результати роботи нової моделі із початковою за показником частки правильних відповідей.

**Для варіантів 3-4:**

Проаналізуйте вплив ознак рівня символів, таких як n-грами символів, на ефективність класифікації за моделлю логістичної регресії. Перевірте гіпотезу про те, що певні шаблони символів вказують на сарказм у тексті.

**Для варіантів 5-6:**

Використайте альтернативний векторизатор зі списку [sklearn.feature_extraction.text](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.feature_extraction.text) для подання тексту в числовий формат. Побудуйте нову модель логістичної регресії та проведіть за нею класифікацію текстів на предмет наявності сарказму. Порівняйте результати класифікації за новою моделлю (з новим векторизатором) з результатами класифікації початкової моделі.

**Для варіантів 7-8:**

Побудуйте та інтерпретуйте шлях регуляризації для моделі логістичної регресії. Використайте цей аналіз, щоб вибрати відповідний рівень регуляризації.

**Для варіантів 9-10:**

Порівняйте побудовану модель логістичної регресії з іншою моделлю (дерево рішень, лінійна регресія, k-найближчих сусідів тощо) за показником частки правильних відповідей. Зробіть висновок щодо ефективності роботи моделей.

**Для варіантів 11-12:**

Використайте техніки урізноманітнення текстових даних ([data augmentation](https://neptune.ai/blog/data-augmentation-in-python)), щоб збільшити розмір і різноманітність навчальних даних. Обговоріть обрані методи та їхній вплив на роботу моделі.

**Для варіантів 13-14:**

Застосуйте такі механізми обробки текстової інформації, як виправлення орфографії чи нормалізація сленгу. Проаналізуйте їх вплив на ефективність моделі логістичної регресії.

**Для варіантів 15-16:**

Визначте та відфільтруйте викиди в наборі даних. Обґрунтуйте обрання методу виявлення викидів та проаналізуйте його вплив на ефективність моделі.

**Для варіантів 17-18:**

Виконайте стратифіковану k-кратну перехресну перевірку (stratified k-fold cross-validation), щоб забезпечити збереження розподілу класів у кожній складці. Проаналізуйте вплив стратифікації на результат моделі.

**Для варіантів 19-20:**

Відкалібруйте результати ймовірностей за допомогою моделі логістичної регресії, використовуючи такі методи, як ізотонічна регресія ([Isotonic regression](https://scikit-learn.org/stable/modules/isotonic.html)) або масштабування Платта ([Platt scaling](https://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html)). Проаналізуйте важливість добре відкаліброваних ймовірностей та їх вплив на ефективність моделі.


<a class="anchor" id="lab-4.5"></a>

## <span style="color:blue; font-size:1.2em;">4.5. Практичне застосування результатів інтелектуального аналізу даних</span>

[Повернутися до змісту](#lab-4)

<span style="color:red; font-size:2em;">Завдання 5</span>

У завданні 5 поміркуйте над застосуванням моделі логістичної регресії (і не тільки) для розв'язання практичних завдань. Проаналізуйте складнощі, що пов’язані з цією проблемою.

**Для варіанту 1:**

Запропонуйте технічні кроки для розгортання побудованої моделі у виробничому середовищі.

**Для варіанту 2:**

Обговоріть стратегії адаптації моделі до роботи з текстом на різних мовах, з огляду на додаткові виклики та складнощі, що із цим пов’язані.

**Для варіанту 3:**

Запропонуйте стратегії налаштування моделі для постійного навчання та вдосконалення на основі відгуків користувачів або нових даних.

**Для варіанту 4:**

Запропонуйте як налаштувати моніторинг розгорнутої моделі та стратегії постійної підтримки моделі.

**Для варіанту 5:**

Оцініть модель логістичної регресії, з огляду на її ефективність, користувацький досвід та етичні міркування, щоб отримати цілісне уявлення про її застосовність і можливість для вдосконалення.

**Для варіанту 6:**

Запропонуйте стратегії, які могли б забезпечити високу постійну точність моделі, з огляду на те, що основні дані змінюються із часом.

**Для варіанту 7:**

Оцініть стійкість моделі до атак зловмисників, коли вхідним текстом навмисно маніпулюють, щоб ввести модель в оману. Проаналізуйте стратегії, щоб зробити модель більш стійкою до таких атак.

**Для варіанту 8:**

Запропонуйте, як можна використати відгуки користувачів для покращення моделі та підвищення її ефективності.

**Для варіанту 9:**

Переконайтеся, що модель і її розгортання відповідають відповідним правовим і регуляторним стандартам, особливо щодо конфіденційності та безпеки даних користувачів.

**Для варіанту 10:**

Запропонуйте нові шляхи для виявлення сарказму в тексті.

**Для варіанту 11:**

Оцініть потенційні упередження в моделі в разі застосування в реальних сценаріях виявлення сарказму та обговоріть стратегії для забезпечення справедливості прийняття рішень.

**Для варіанту 12:**

Обговоріть майбутні тенденції та нові технології, які можуть вплинути на сферу класифікації текстів і виявлення сарказму. Запропонуйте шляхи інтеграції цих досягнень у поточний робочий процес.

**Для варіанту 13:**

Оцініть стійкість моделі до можливих негативних зовнішніх впливів.

**Для варіанту 14:**

Виділіть як мінімум 5 етичних проблем, що пов’язані із застосуванням моделі в реальних сценаріях.

**Для варіанту 15:**

Обговоріть важливість пояснюваності та прозорості моделі, особливо в тих програмних застосунках, де користувачі взаємодіють із моделлю. Запропонуйте шляхи покращення цих аспектів.

**Для варіанту 16:**

Обговоріть, які показники ефективності є найбільш важливими під час практичного використання моделей інтелектуального аналізу даних (і чому вони такими є), з огляду на конкретний контекст виявлення сарказму.

**Для варіанту 17:**

Оцініть масштабованість моделі та обговоріть будь-які необхідні механізми оптимізації для ефективної роботи в реальних умовах.

**Для варіанту 18:**

Проаналізуйте потенційні проблеми безпеки, що пов’язані з розгортанням моделі, і запропонуйте стратегії для їхнього вирішення.

**Для варіанту 19:**

Проаналізуйте стратегії роботи з двозначністю та важливість контексту для виявлення сарказму.

**Для варіанту 20:**

Проаналізуйте можливість розширення моделі для включення інших модальностей (наприклад, зображення, аудіо) для виявлення саркастичних текстів.
